# Benchmarking

This notebook explores a key constructor parameter that affects the `FileSenderClient` performance: `concurrent_chunks`. `concurrent_chunks` sets a limit on the number of chunks of file data that can loaded into memory at the same time. Therefore a lower number for this parameter is expected to reduce memory usage but also increase runtime.

The built-in `benchmark` function can be used to run the comparison. The below code benchmarks an upload using 3 files, each of which is 100 MB. We test a read limit of 1-10 and a request limit of 1-10.

In [1]:
from filesender.benchmark import benchmark, make_tempfiles
from os import environ

with make_tempfiles(size=100_000_000, n=3) as paths:
    results = benchmark(paths, limit=6, apikey=environ["API_KEY"], base_url=environ["BASE_URL"], recipient=environ["RECIPIENT"], username=environ["USERNAME"])

tmpe10e9bu9: 100%|██████████| 8/8 [00:01<00:00,  4.42it/s]


Transform the results into a data frame for analysis:

In [3]:
import pandas as pd
result_df = pd.DataFrame.from_records(vars(result) for result in results)
result_df.memory = result_df.memory / 1024 ** 2
result_df

,time,memory,concurrent_chunks
0,7.708177,0.195992,1
1,14.282335,0.350613,2
2,9.678596,0.401077,3
3,9.738843,0.406414,4
4,7.315477,0.418480,5


The memory usage consistently increases as we increase the number of concurrent chunks, as expect.
What is more unusual is that the time doesn't follow a consistent pattern in relation to the number of chunks